In [1]:
import numpy as np
import pandas as pd
import regex as re
import json
import matplotlib.pyplot as plt
import glob
import time
from IPython import display

# suppress data caveat warning
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
%cd /content/drive/MyDrive/de_project_wine_recommender/FeelFlask/backend/data_works

/content/drive/MyDrive/de_project_wine_recommender/FeelFlask/backend/data_works


In [3]:
alcohol_data = pd.read_pickle('./preprocessed_alcohol_v4.pkl')

In [4]:
with open('../flavor_update.json', 'r') as fr:
  flavor_info = json.load(fr)
# flavor_info

In [5]:
# 재료 레시피 인덱스에 따라 정렬이 되도록 합니다.
for i in range(len(alcohol_data)):
  alcohol_data['Ingredients'][i] = sorted(alcohol_data['Ingredients'][i], key=lambda x : x[1])
for i in range(len(alcohol_data)):
  alcohol_data['Measures'][i] = sorted(alcohol_data['Measures'][i], key=lambda x : x[1])

In [6]:
# 어떤 재료를 사용했는지를 나타내는 벡터를 생성하고,
# 얼마만큼 사용했는지에 대한 벡터가 정의 되어야 합니다.

def find_index(name):
  for i in range(len(flavor_info)):
    if name == flavor_info[i]['name']:
      return i
  print("Non Exist Ingredients!")
  print(name)
  return None

def delete_ap(name):
  name = re.sub('\'', '', name)
  return name

ingredient_data = alcohol_data['Ingredients']
measurement_data = alcohol_data['Measures']
ing_set = []
for row in range(len(ingredient_data)):
  for ing_idx in range(len(ingredient_data[row])):
    ing_set.append(ingredient_data[row][ing_idx][0])

unique_ingredient = np.unique(ing_set)

# ingrident 요소들을 integer 값으로 맵핑합니다.
ing_mapper = {}
for i in range(len(unique_ingredient)):
    ing_mapper[unique_ingredient[i]] = i+1
print(ing_mapper)
int_to_ing = {value: key for key, value in ing_mapper.items()}

# ingredient label for multi classification problem
ing_label = []
for row in range(len(ingredient_data)):
  ing_label.append([])
  for ing_index in range(len(ingredient_data[row])):
    ing_label[row].append(ing_mapper[ingredient_data[row][ing_index][0]])

# measurement label for regression problem
measure_label = []
for row in range(len(measurement_data)):
  measure_label.append([])
  for measure_index in range(len(measurement_data[row])):
    measure_label[row].append(float(measurement_data[row][measure_index][0]))


def decode(array, name_list, ingredient_length = 5):
  row, col = array.shape
  prediction = []
  for i in range(row):
    prediction.append({})

    for j in range(col):
      if array[i][j] > 0:
        prediction[i][int_to_ing[j]] = array[i][j]

  for recipe_index in range(len(prediction)):
    prediction[recipe_index] = sorted(prediction[recipe_index].items(), key=lambda x: x[1], reverse=True)

  output = []
  for recipe_index in range(len(prediction)):
    output.append(prediction[recipe_index][0:ingredient_length])
  for output_index in range(len(output)):
    output[output_index].append(('name', name_list[output_index]))
  return output

{'151 proof rum': 1, '7-up': 2, 'absinthe': 3, 'absolut citron': 4, 'absolut kurant': 5, 'absolut peppar': 6, 'absolut vodka': 7, 'advocaat': 8, 'agave syrup': 9, 'ale': 10, 'allspice': 11, 'almond': 12, 'almond flavoring': 13, 'amaretto': 14, 'angelica root': 15, 'angostura bitters': 16, 'anis': 17, 'anise': 18, 'anisette': 19, 'aperol': 20, 'apfelkorn': 21, 'apple': 22, 'apple brandy': 23, 'apple cider': 24, 'apple juice': 25, 'apple schnapps': 26, 'applejack': 27, 'apricot': 28, 'apricot brandy': 29, 'aquavit': 30, 'asafoetida': 31, 'añejo rum': 32, 'bacardi limon': 33, "bailey's irish cream": 34, 'baileys irish cream': 35, 'banana': 36, 'banana liqueur': 37, 'beer': 38, 'benedictine': 39, 'berries': 40, 'bitter lemon': 41, 'bitters': 42, 'black pepper': 43, 'black sambuca': 44, 'blackberry brandy': 45, 'blackcurrant cordial': 46, 'blackcurrant squash': 47, 'blended whiskey': 48, 'blue curacao': 49, 'blueberry schnapps': 50, 'bourbon': 51, 'brandy': 52, 'brown sugar': 53, 'butter': 

In [ ]:
from tensorflow.keras.utils import pad_sequences
ing_label = pad_sequences(ing_label, padding='post')

for i in range(len(measure_label)):
  for j in range(12 - len(measure_label[i])):
    measure_label[i].append(0)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# measure_label = scaler.fit_transform(measure_label)

In [ ]:
# change to numpy array
measure_label = np.array(measure_label)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Embedding, Flatten, Embedding, BatchNormalization, Dropout, ReLU, Add, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, AdamW
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import os

In [ ]:
BUFFER_SIZE = 546
BATCH_SIZE = 546

dataset_ing = tf.data.Dataset.from_tensor_slices(ing_label).batch(BATCH_SIZE)
dataset_mas = tf.data.Dataset.from_tensor_slices(measure_label).batch(BATCH_SIZE)
# for element in dataset:
#   for item in element:
#     print(item)

In [ ]:
# GAN : 생성자는 랜덤한 노이즈로 부터 가짜 레시피를 만들어 낼 겁니다.
# 그러면 판별자는 실제 레시피와 GAN이 생성한 레시피를 입력으로 받아
# 이 레시피가 실제 존재하는지에 대한 여부를 출력할 것입니다.

def make_generator_model():
  generator_ing = tf.keras.layers.Input(shape=(12)) # 명목값
  generator_mas = tf.keras.layers.Input(shape=(12)) # 수치값

  h_m = tf.keras.layers.Dense(64)(generator_mas)
  h_m = tf.keras.layers.ReLU()(h_m)
  h_m = tf.keras.layers.Dropout(0.25)(h_m)
  h_m = tf.keras.layers.Dense(128)(generator_mas)
  h_m = tf.keras.layers.ReLU()(h_m)

  embedding = tf.keras.layers.Embedding(309, 32)(generator_ing)
  flatten = tf.keras.layers.Flatten()(embedding)
  concat = tf.keras.layers.Concatenate()([h_m, flatten])
  h1 = tf.keras.layers.Dense(512)(flatten)
  r1 = tf.keras.layers.ReLU()(h1)

  output_ing = tf.keras.layers.Dense(12, activation='sigmoid')(r1) # 12개의 클래스에 대해서, multi-label classification을 수행합니다. 이때 12개는 입력한 레시피 재료 집합에 한정됩니다.
  output_mas = tf.keras.layers.Dense(12)(r1) # 12개의 값을 예측하도록 합니다. 이중에 처음 몇개만 유효한 값을 갖습니다.

  model = Model(inputs=[generator_ing, generator_mas], outputs=[output_ing, output_mas])
  return model


def make_discriminator_model():
  dis_ing = tf.keras.layers.Input(shape=(12, ))
  dis_mas = tf.keras.layers.Input(shape=(12, ))

  h1 = tf.keras.layers.Dense(448)(dis_ing)

  h_m = tf.keras.layers.Dense(32)(dis_mas)
  h_m = tf.keras.layers.Dropout(0.25)(h_m)
  h_m = tf.keras.layers.ReLU()(h_m)

  h_m = tf.keras.layers.Dense(64)(h_m)
  h_m = tf.keras.layers.ReLU()(h_m)

  concat = Concatenate()([h1, h_m])
  h1 = tf.keras.layers.Dense(512)(concat)
  r1 = tf.keras.layers.ReLU()(h1)

  output = tf.keras.layers.Dense(1)(r1)

  model = Model(inputs=[dis_ing, dis_mas], outputs=output)
  return model



In [ ]:
generator = make_generator_model()
discriminator = make_discriminator_model()

noise1 = np.array([35, 107, 129, 1, 0, 0, 0, 0, 0, 0, 0, 0]).reshape(1, -1)
noise2 = np.array([120, 100, 24, 52, 0, 0, 0, 0, 0, 0, 0, 0]).reshape(1, -1)

generated_recipe = generator([noise1, noise2], training=False)

decision = discriminator(generated_recipe)

print(generated_recipe)
print(decision) # 0.5 이상이면 real, 0.5 이하이면 fake

[<tf.Tensor: shape=(1, 12), dtype=float32, numpy=
array([[0.49466804, 0.5099499 , 0.5198582 , 0.50499326, 0.5010081 ,
        0.49757108, 0.51188916, 0.504326  , 0.49787697, 0.5006898 ,
        0.4959364 , 0.5043837 ]], dtype=float32)>, <tf.Tensor: shape=(1, 12), dtype=float32, numpy=
array([[-0.01077114, -0.05570974, -0.01942833, -0.03228064, -0.00861003,
         0.00313198,  0.00736312,  0.00053492, -0.00087288,  0.06138745,
         0.02269318, -0.04363787]], dtype=float32)>]
tf.Tensor([[-0.01521248]], shape=(1, 1), dtype=float32)


In [ ]:
# helper function to calculate cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# 판별자는 binary classification을 수행하며, 해당 입력이 실제 있는 입력이었는지, 거짓이었는 지를 구분하는
# 일을 수행합니다. 실제 있는 샘플에 대해서는 값을 1로, 없는 샘플에 대해서는 0으로 출력합니다.
def discriminator_loss(real_output, fake_output):
  real_loss = cross_entropy(tf.ones_like(real_output),real_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  total_loss = real_loss + fake_loss
  return total_loss

def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
checkpoint_dir = './checkpoint/gan/'
checkpoint_prefix = os.path.join(checkpoint_dir, 'GAN')
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
noise_dim=12 # 12개의 항목에 대해서 노이즈를 만들어야 합니다.

In [ ]:
max(measure_label.flatten()) # 최대 값은 4000입니다.

4000.0

In [ ]:
tf.convert_to_tensor(np.random.randint(0, 309, (10, 12))) # 308까지 유효합니다.

<tf.Tensor: shape=(10, 12), dtype=int64, numpy=
array([[ 60, 302, 120, 169, 235,   3,  13,  69,  67, 150, 176, 307],
       [300,  99, 261,  12, 106,  38, 302, 163,   1, 126,  73,  65],
       [216, 181,  70, 266,  39, 123,  19, 170, 130, 105, 264, 262],
       [ 33,  83, 163,  55, 142,  42, 259, 222, 102, 116, 276, 206],
       [210, 147, 272, 288,  80, 124, 145,  98, 141, 235,  54, 268],
       [146, 300, 276,  83, 235,  38, 258, 153,  95,  68, 111, 137],
       [306, 125,  66, 286, 307, 162,  20,  69,  55,  21,  26, 303],
       [216,  75,   4, 233, 278, 285, 142, 231, 304,  31, 106, 118],
       [123,  95, 289, 238, 108, 216,  66, 156, 262, 259,  79, 270],
       [279, 230, 198, 236, 200,  81,  83, 308, 164, 194, 197,  70]])>

In [ ]:
(np.random.random((3, 12)) * 4001)

array([[3594.18343909,    8.47559276,  593.97203128,  490.70746409,
        1748.59827724, 2585.11803085, 2359.38033012, 3248.18529384,
        1155.6081032 , 1901.11800276,  514.61289966, 3320.03663425],
       [2141.78941521,  173.34639202, 3859.68434287, 1160.47757386,
        1823.91643141, 3255.55885165, 2843.00508709, 2482.76203883,
        1375.21643874,  216.10861067, 3039.41999597, 1597.88637598],
       [2696.59571569, 2509.87022847, 1439.5035138 , 3657.26367059,
        3254.97728442, 2762.76121014,  562.71578247,  262.00668414,
        2203.03091584, 1545.07776131, 2127.32613401, 3302.74439048]])

In [ ]:
@tf.function
def train_step(recipes):
  noise_mas = tf.convert_to_tensor(np.random.random((BATCH_SIZE, noise_dim)) * 4001)
  noise_ing = tf.convert_to_tensor(np.random.randint(0, 309, (BATCH_SIZE, noise_dim)))

  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    generated_recipe = generator([noise_ing, noise_mas], training=True)

    real_output = discriminator([recipes[0], recipes[1]], training=True)
    fake_output = discriminator(generated_recipe, training=True)

    gen_loss = generator_loss(fake_output)
    disc_loss = discriminator_loss(real_output, fake_output)

  gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
  gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


In [ ]:
num_examples_to_generate = 16
seed_ing = tf.convert_to_tensor(np.random.randint(0, 309, (num_examples_to_generate, noise_dim)))
seed_mas = tf.convert_to_tensor(np.random.random((num_examples_to_generate, noise_dim)) * 4001)
seed = [seed_ing, seed_mas]

In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()
    train_step(dataset)

    # 15 epochs가 지날 때마다 모델을 저장합니다.
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    # print(' 에포크 {} 에서 걸린 시간은 {} 초 입니다'.format(epoch+1, time.time()-start))
    print('Time for epoch {} is {} sec'.format(epoch+1, time.time()-start))

  # 마지막 에포크가 끝난 후 생성합니다.
  display.clear_output(wait=True)
  generate_and_save_recipes(generator, epochs, seed)


In [ ]:
def generate_and_save_recipes(model, epoch, test_input):
  predictions = model(test_input, training=False)
  print(predictions)

In [ ]:
%%time
EPOCHS=750
train([ing_label, measure_label], EPOCHS)

[<tf.Tensor: shape=(16, 12), dtype=float32, numpy=
array([[9.3398124e-01, 9.9992740e-01, 9.9985397e-01, 4.8239496e-01,
        5.4624289e-02, 1.5976483e-02, 2.6695861e-03, 7.5880796e-01,
        3.6750705e-04, 1.3670146e-04, 3.2099494e-04, 4.8756971e-05],
       [9.3236613e-01, 9.9993098e-01, 9.9985713e-01, 4.8748475e-01,
        5.5368647e-02, 1.6212232e-02, 2.5907354e-03, 7.6028323e-01,
        4.0756405e-04, 1.3709346e-04, 3.4401898e-04, 4.7946978e-05],
       [9.1903782e-01, 9.9986058e-01, 9.9973339e-01, 4.8792264e-01,
        6.7613848e-02, 2.1081526e-02, 3.8591195e-03, 7.3574126e-01,
        6.6782092e-04, 2.4478557e-04, 5.9488125e-04, 9.5425108e-05],
       [9.3383187e-01, 9.9994087e-01, 9.9987769e-01, 4.9023932e-01,
        5.2876886e-02, 1.5411377e-02, 2.3285833e-03, 7.5442201e-01,
        3.0772228e-04, 1.1737511e-04, 2.8024675e-04, 4.1149331e-05],
       [9.3540984e-01, 9.9992734e-01, 9.9985427e-01, 4.7823307e-01,
        5.0585225e-02, 1.8534217e-02, 2.5517426e-03, 7.574985

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
noise_mas = tf.convert_to_tensor(np.random.random((1, noise_dim)) * 4001)
noise_ing = tf.convert_to_tensor(np.random.randint(0, 309, (1, noise_dim)))


generated_recipe = generator([noise_ing, noise_mas], training=False)

decision_fake = discriminator(generated_recipe, training=False)
decision_real = discriminator([ing_label[0:1], measure_label[0:1]], training=False)



print("Generated Recipe : ", generated_recipe)
print("Decision Fake: ", decision_fake)
print("Decision Real: ", decision_real)

Generated Recipe :  [<tf.Tensor: shape=(1, 12), dtype=float32, numpy=
array([[9.2807782e-01, 9.9991167e-01, 9.9983501e-01, 4.8020908e-01,
        5.7478085e-02, 1.8261196e-02, 2.7520612e-03, 7.5668401e-01,
        4.3593196e-04, 1.6356028e-04, 3.7337648e-04, 5.9637627e-05]],
      dtype=float32)>, <tf.Tensor: shape=(1, 12), dtype=float32, numpy=
array([[ 4.615552  ,  4.1480546 ,  4.5051126 , 14.573082  ,  9.656284  ,
         1.1486702 ,  0.9760436 , -2.1578417 ,  2.992358  , -3.6636682 ,
         0.02239071, -6.125287  ]], dtype=float32)>]
Decision Fake:  tf.Tensor([[-3.0637462]], shape=(1, 1), dtype=float32)
Decision Real:  tf.Tensor([[85.84105]], shape=(1, 1), dtype=float32)
